In [1]:
import random
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import quad
from time import time
import warnings
ok = "[ \033[1m\033[92m OK \033[0m\033[0m ]"
fail = "[\033[1m\033[91mFAILED\033[0m\033[0m]"

In [2]:
def E(z, param):
    H1= 1 + param[0]
    H2= -pow(param[0],2) + param[1]
    H3= 3*pow(param[0],2)*(1+param[0]) - param[1]*(3+4*param[0]) - param[2]
    H4= -3*pow(param[0],2)*(4+8*param[0]+5*pow(param[0],2)) + param[1]*(12+32*param[0]+25*pow(param[0],2)-4*param[1]) + param[2]*(8+7*param[0]) + param[3]

    Q1= (-6*H1*H4 + 12*H2*H3)/(24*H1*H3 - 36*H2**2)
    Q2= (3*H2*H4 - 4*H3**2)/(24*H1*H3 - 36*H2**2)
    P0= 1
    P1= H1 + Q1
    P2= H2/2 + Q1*H1 + Q2

    EE= (P0 + P1*z + P2*z**2)/(1 + Q1*z + Q2*z**2)
    return 1/EE

def mu(z, param):
    muu= 5*np.log10((1+z)*quad(E, 0, z, args=param)[0])
    return muu

In [3]:
class Chromosome:
    def __init__(self, param=None):
        self.param = param
        if self.param is None:
            self.param = [
                random.uniform(-10, 0),
                random.uniform(0, 20),
                random.uniform(-20, 20),
                random.uniform(-15, 30)
            ]
    
    def xi2(self, z, error):
        A= 0
        B= 0
        C= 0
        for i in range(len(z)):
            A+= pow(mu(z[i], self.param)-muu[i], 2)/pow(error[i], 2)
            B+= (mu(z[i], self.param)-muu[i])/pow(error[i], 2)
            C+= 1/pow(error[i], 2)
            x2= A- pow(B,2)/C
        return x2
    
    def get_param(self):
        return self.param
    
    def fittness(self, z, error):
        self.ft = np.abs(1044 - self.xi2(z, error))
        return self.ft
    
    def mutate(self, ranges, active, possibility=0.005):
        for i in range(4):
            if (i in active) and (possibility >= random.randint(0, 1000)/1000):
                k = random.uniform(-2, 2)
                self.param[i] += random.uniform(ranges[i][0]*k, ranges[i][1]*k)
                
    def breed(self, mate, active):
        child1 = Chromosome([
            (np.mean((self.get_param()[i], mate.get_param()[i]))) if i in active else self.get_param()[i] for i in range(4)
        ])
        
        child2 = Chromosome([
            (np.mean((self.get_param()[i], mate.get_param()[i]))) if i in active else mate.get_param()[i] for i in range(4)
        ])
            
        return child1, child2

In [4]:
class Genetics:
    def __init__(
        self,
        z,  # data points z
        error,  # data points error
        population=1000,  # population
        max_iter=800,  # max number of generations
        pb=0.80,  # breeding probability
        pm=0.005,  # mutation probability
        min_fittness=0
    ):
        
        self.z = z
        self.error = error
        
        self.generation = 1
        self.population_count = population
        self.max_iter = max_iter
        self.pb = pb
        self.pm = pm
        self.min_fittness = min_fittness
        
        self.avg_fittness = []  # stores average fittness value per generation
        self.best_fittness = []  # stores best fittness value per generation
        
        self.population = [Chromosome() for c in range(self.population_count)]
        print(ok, 'Generated initial population')
        
        self.fitt = []
        
        with warnings.catch_warnings():
            warnings.simplefilter('error')
            for p in range(self.population_count):
                first = True
                while True:
                    try:
                        self.fitt.append(self.population[p].fittness(self.z, self.error))
                        break
                    except:
                        if first:
                            print(fail, f'Chromosome {p} is corrupted, attempting replacement')
                            first = False
                        self.population[p] = Chromosome()
        
        print(ok, 'Calculated fittness value for all chromosomes')
        
        self.final_ans = self.population[0]
        self.final_ans_ft = self.fitt[0]
        
        self.save_stats()
        print(ok, 'Stats saved')
        
        self.active = [0, 1]
        
    def save_stats(self):
        sigma = 0
        best = self.population[0]
        bf = self.fitt[0]
        for i in range(self.population_count):
            c = self.population[i]
            cf = self.fitt[i]
            sigma += cf
            if cf < bf:
                best = c
                bf = cf
        self.avg_fittness.append(sigma/self.population_count)
        self.best_fittness.append(bf)
        
        if self.final_ans_ft > bf:
            self.final_ans = best
            self.final_ans_ft = bf
            
    def select_parents(self):
        parents = [(i, self.fitt[i]) for i in range(self.population_count)]
        parents.sort(key=lambda a: a[1])
        parents = parents[:int(self.population_count/2)]
        parents = [self.population[p[0]] for p in parents]
        return parents*2
    
    def reproduce(self, parents):
        childes = []
        a = 0
        while a + 1 < len(parents):
            r = random.randint(0, self.population_count)
            if r >= self.pb * self.population_count:
                childes.append(Chromosome(parents[a].get_param()))
                childes.append(Chromosome(parents[a+1].get_param()))
            else:
                childes.extend(parents[a].breed(parents[a+1], self.active))
            a += 2
        return childes
    
    def next_gen(self):
        
        if self.generation%20 == 0:
            self.active = [
                self.active[0] - 2*(self.active[0]/2) + 2*((-3*self.active[0]/2)+self.active[1]),
                self.active[1] - 2*(self.active[0]/2) + 2*((-3*self.active[0]/2)+self.active[1])
            ]
        
        parents = self.select_parents()
        print(ok, 'Parents selected')
        
        childes = self.reproduce(parents)
        print(ok, 'Offsprings created')
        
        qs = []
        js = []
        ss = []
        ls = []
        
        for c in childes:
            qs.append(c.get_param()[0])
            js.append(c.get_param()[1])
            ss.append(c.get_param()[2])
            ls.append(c.get_param()[3])
                
        ranges = [
            [min(qs), 0],
            [0, max(js)],
            [min(ss), max(ss)],
            [min(ls), max(ls)]
        ]
        print(ok, 'Mutation range calculated')

        self.fitt = []
        
        with warnings.catch_warnings():
            warnings.simplefilter('error')
            for c in range(self.population_count):
                first = True
                temp = Chromosome(childes[c])
                temp.mutate(ranges, self.active, possibility=self.pm)
                while True:
                    try:
                        self.fitt.append(temp.fittness(self.z, self.error))
                        childes[c] = Chromosome(temp.get_param())
                        break
                    except:
                        if first:
                            print(fail, f'Offspring {c} is corrupted, attempting mutation')
                            first = False
                        temp = Chromosome(childes[c])
                        temp.mutate(ranges, self.active, possibility=self.pm)
            
        self.population = childes
        print(ok, 'New generation successfully generated')
        
        self.save_stats()
        print(ok, 'Stats saved')
        self.generation += 1
        
    def terminate(self):
        if self.min_fittness >= self.best_fittness[-1]:
            print(ok, 'Achieved optimal fittness value')
            return True
        
        return self.generation >= self.max_iter

In [5]:
data= np.genfromtxt('mock_pantheon.txt')
z1= data[:,0]
muu= data[:,1]
error= data[:,2]

In [6]:
population = 20
max_iter = 40
pb = 0.80
pm = 0.5
min_fittness = 0

In [7]:
a = time()
x = Genetics(
    z=z1,
    error=error,
    population=population,
    max_iter=max_iter,
    pb=pb,
    pm=pm,
    min_fittness=min_fittness   
)

print("Gen", "\tBest", "\t\t\tAvg")

while not x.terminate():
    print(
        x.generation, "\t"+str(x.best_fittness[-1]),
        "\t"+str(x.avg_fittness[-1])
    )
    x.next_gen()

b = time()
print("Time:", round(b-a, 2), "seconds")
print(x.final_ans.get_param())

[  OK  ] Generated initial population
[FAILED] Chromosome 0 is corrupted, attempting replacement
[FAILED] Chromosome 1 is corrupted, attempting replacement
[FAILED] Chromosome 3 is corrupted, attempting replacement
[FAILED] Chromosome 4 is corrupted, attempting replacement
[FAILED] Chromosome 6 is corrupted, attempting replacement
[FAILED] Chromosome 10 is corrupted, attempting replacement
[FAILED] Chromosome 11 is corrupted, attempting replacement
[FAILED] Chromosome 12 is corrupted, attempting replacement
[FAILED] Chromosome 14 is corrupted, attempting replacement
[FAILED] Chromosome 19 is corrupted, attempting replacement
[  OK  ] Calculated fittness value for all chromosomes
[  OK  ] Stats saved
Gen 	Best 			Avg
1 	167.7731496989727 	16460.91048971936
[  OK  ] Parents selected
[  OK  ] Offsprings created
[  OK  ] Mutation range calculated
[FAILED] Offspring 7 is corrupted, attempting mutation
[FAILED] Offspring 11 is corrupted, attempting mutation
[  OK  ] New generation successful

[  OK  ] New generation successfully generated
[  OK  ] Stats saved
16 	75.00643754005432 	1085.6111014798284
[  OK  ] Parents selected
[  OK  ] Offsprings created
[  OK  ] Mutation range calculated
[FAILED] Offspring 7 is corrupted, attempting mutation
[FAILED] Offspring 14 is corrupted, attempting mutation
[  OK  ] New generation successfully generated
[  OK  ] Stats saved
17 	39.21591031551361 	3373.948658286035
[  OK  ] Parents selected
[  OK  ] Offsprings created
[  OK  ] Mutation range calculated
[FAILED] Offspring 12 is corrupted, attempting mutation
[FAILED] Offspring 13 is corrupted, attempting mutation
[  OK  ] New generation successfully generated
[  OK  ] Stats saved
18 	64.48755958676338 	1505.4352511368693
[  OK  ] Parents selected
[  OK  ] Offsprings created
[  OK  ] Mutation range calculated
[FAILED] Offspring 3 is corrupted, attempting mutation
[FAILED] Offspring 4 is corrupted, attempting mutation
[FAILED] Offspring 10 is corrupted, attempting mutation
[FAILED] Offspr

In [ ]:
a = Chromosome()

In [ ]:
ranges = [
    [-50, 0],
    [0, 50],
    [-50, 50],
    [-50, 50]
]

active = [0, 1]
pm = 0.5

In [ ]:
a.mutate(ranges, active, possibility=pm)
a.get_param()

In [ ]:
a.fittness(z1, error)

In [ ]:
random.randint(0, 1)